# Location Finder
An excited new technology in the field of image datascience and radiology is the use of machine learning to locate and idenpngy specific structures in medical images in 3D space. This technology can be used to assist radiologists in diagnosing diseases and conditions, as well as to help researchers analyze large volumes of medical image data.

An example of this technology is the nnU-Net framework (linke to their nature publication: https://www.nature.com/articles/s41592-020-01008-z), which is a deep learning model for medical image segmentation. This model can be used to locate and idenpngy different organs in medical images in 3D space. In this notebook, we will demonstrate how to use a nnU-Net model trained across 120 abdomen CT and 40 MRI scans to locate and idenpngy structures in your image stack from either image technology.

The output of the model will be a prediction on the location of the structures in the image stack. The prediction will be in the form of a description that describes the location of the structures in the image stack. The prediction will be displayed in the notebook.

This notebook will guide you through the process of downloading the required files, uploading your image stack, formating your uploaded image stack and running the model to generate a prediction. To get started, run the cells below and follow the instructions.

# Download required files
Before we can use the model, we need to download the required files. The files include the trained model weights and configuration. The files are stored in a ZIP archive, which we will download and automatically extract. When the extraction is complete, the script will automatically set the environment variables required by the prediction logarithm nnU-Net framework.

In [3]:
import os
import zipfile

!pip install ipywidgets nnUNetv2 requests

import requests
# Define the URL and output file name
url = "https://wustl.box.com/shared/static/12gz6cy9n5isi5e3b6zcxgow6g9uf9v8.zip"
output_file = "Results_Folder.zip"
extracted_folder = "Results_Folder"

# Define the URL and file paths
url = "https://wustl.box.com/shared/static/12gz6cy9n5isi5e3b6zcxgow6g9uf9v8.zip"
output_file = "Results_Folder.zip"
extracted_folder = "Results_Folder"

# Download the file using requests
def download_file(url, output_file):
    print("Starting download...")
    with requests.get(url, stream=True) as response:
        response.raise_for_status()  # Raise an exception for HTTP errors
        with open(output_file, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
    print(f"File downloaded successfully: {output_file}")

# Unzip the downloaded file
def unzip_file(zip_file, extract_to):
    print(f"Unzipping file: {zip_file}")
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Files extracted to: {extract_to}")

# Perform the download and extraction
if os.path.exists(extracted_folder):
    # Delete existing files
    for root, dirs, files in os.walk(extracted_folder, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
    os.rmdir(extracted_folder)
    print(f"Deleted existing folder: {extracted_folder}")

if os.path.exists(output_file):
    os.remove(output_file)
    print(f"Deleted existing file: {output_file}")

download_file(url, output_file)
unzip_file(output_file, extracted_folder)


# Get the absolute path of the extracted folder
extracted_path = os.path.abspath(extracted_folder)

extracted_path = os.path.join(extracted_path, "Results_Folder")
# Set environment variables
os.environ['nnUNet_raw'] = extracted_path
os.environ['nnUNet_results'] = extracted_path
os.environ['nnUNet_preprocessed'] = extracted_path

# Print the environment variables to verify
print(f"nnUNet_raw={os.environ['nnUNet_raw']}")
print(f"nnUNet_results={os.environ['nnUNet_results']}")
print(f"nnUNet_preprocessed={os.environ['nnUNet_preprocessed']}")

  Obtaining dependency information for argparse from https://files.pythonhosted.org/packages/f2/94/3af39d34be01a24a6e65433d19e107099374224905f1e0cc6bbe1fd22a2f/argparse-1.4.0-py2.py3-none-any.whl.metadata
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Starting download...


/Users/alexandergadin/PycharmProjects/QP2_Big_data_project_tools/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


File downloaded successfully: Results_Folder.zip
Unzipping file: Results_Folder.zip
Files extracted to: Results_Folder
nnUNet_raw=/Users/alexandergadin/PycharmProjects/QP2_Big_data_project_tools/Results_Folder/Results_Folder
nnUNet_results=/Users/alexandergadin/PycharmProjects/QP2_Big_data_project_tools/Results_Folder/Results_Folder
nnUNet_preprocessed=/Users/alexandergadin/PycharmProjects/QP2_Big_data_project_tools/Results_Folder/Results_Folder


# Upload the image stack
Upload and select the folder containing the image stack you want to analyze. The image stack should be in a folder containing the image files. You can upload files to EIT by going to the filebrowser tab at [linuxlab.engr.wust.edu](https://linuxlab.engr.wustl.edu) and clicking the blue upload button. Once you have uploaded the image stack, run the cell below and select the folder containing the image stack.

In [4]:
import os
import ipywidgets as widgets
from IPython.display import display

# Initialize the current path
current_path = os.getcwd()  # This will hold the current directory path

# Create widgets
path_display = widgets.Text(
    value=current_path,
    description='Path:',
    layout=widgets.Layout(width='800px')
)

output = widgets.Output()

def display_directories(path):
    """Display only directories in the given path as clickable rectangles."""
    items = [item for item in os.listdir(path) if os.path.isdir(os.path.join(path, item))]
    buttons = []

    for item in items:
        item_path = os.path.join(path, item)
        button = widgets.Button(
            description=item,
            layout=widgets.Layout(width='auto', height='30px')
        )
        button.style.button_color = '#007bff'
        button.on_click(lambda b, p=item_path: handle_directory_click(p))
        buttons.append(button)

    return widgets.VBox(buttons)

def count_png_files(path):
    """Count the number of .png files in the given directory."""
    return len([f for f in os.listdir(path) if f.endswith('.png')])

def handle_directory_click(directory_path):
    """Handle clicks on directories."""
    global current_path  # Make current_path accessible globally
    current_path = directory_path  # Update global variable
    path_display.value = current_path
    refresh_directory_view(current_path)

    # Check for .png files and display a warning if there are less than two
    png_count = count_png_files(directory_path)
    output.clear_output()
    with output:
        if png_count <= 1:
            print(f"Warning: The directory '{os.path.basename(directory_path)}' contains {png_count} .png file(s).")
        else:
            print(f"The directory '{os.path.basename(directory_path)}' contains enough ({png_count}) .png file(s).")

def navigate_to_parent_directory(_):
    """Navigate to the parent directory."""
    global current_path  # Make current_path accessible globally
    parent_path = os.path.dirname(current_path)
    current_path = parent_path  # Update global variable
    path_display.value = current_path
    refresh_directory_view(current_path)

def refresh_directory_view(path):
    """Refresh the directory view."""
    directory_view.children = [display_directories(path)]

# Button to go to the parent directory
parent_button = widgets.Button(
    description='Go to Parent Directory',
    layout=widgets.Layout(width='auto', height='30px')
)
parent_button.style.button_color = '#007bff'
parent_button.on_click(navigate_to_parent_directory)

# Initialize the directory view
directory_view = widgets.VBox(children=[display_directories(current_path)])

# Layout the widgets
navigation_box = widgets.VBox([path_display, parent_button, directory_view, output])
display(navigation_box)

# If the current path stops updating, run this cell again

# Format the image stack
Using the folder path selected above, we will format the image stack to be compatible with the nnU-Net framework. The formatting will include converting the image stack to .nii.gz format and saving it to a temporary folder. The image stack has to be in .nii.gz format for the nnU-Net framework to process it as that is the same format the model was trained on. Fortunately, it is easy to convert the image stack to .nii.gz format  using the cell below

In [5]:
import os
import tempfile
import nibabel as nib
import numpy as np
from PIL import Image
def count_png_file_s(path):
    """Count the number of .png files in the given directory."""
    return len([f for f in os.listdir(path) if f.endswith('.png')])
# Check if directory_path is set and valid
if 'current_path' not in locals() or not os.path.isdir(current_path) or not count_png_file_s(current_path):
    print("Warning: 'input_path' is not invalid. Please run the previous step to set a valid path to folder with .png files.")
else:
    # Create a temporary folder
    temp_folder = tempfile.mkdtemp()

    # Gather all PNG files in the directory
    png_files = sorted([f for f in os.listdir(current_path) if f.endswith('.png')])

    if not png_files:
        print("No PNG files found in the specified directory.")
    else:
        # Read PNG stack and convert to numpy array
        png_stack = []
        for png_file in png_files:
            img_path = os.path.join(current_path, png_file)
            img = Image.open(img_path).convert('L')  # Convert to grayscale
            png_stack.append(np.array(img))

        # Create a 3D numpy array
        png_stack_array = np.stack(png_stack, axis=-1)

        # Convert the numpy array to NIfTI format
        nifti_img = nib.Nifti1Image(png_stack_array, affine=np.eye(4))

        # Save the NIfTI file as .nii.gz in the temporary folder
        output_file_path = os.path.join(temp_folder, "amos_0000.nii.gz")
        nib.save(nifti_img, output_file_path)

        # Clear other contents in the temp folder, if any
        for file in os.listdir(temp_folder):
            if file != "amos_0000.nii.gz":
                os.remove(os.path.join(temp_folder, file))

        # Save the temp folder path to input_folder

        input_folder = temp_folder
        os.environ['INPUT_FOLDER'] = input_folder
        print(f"Converted PNG stack saved as .nii.gz file in temporary folder: {input_folder}")

/Users/alexandergadin/Downloads/Group_6/MRI_2
Converted PNG stack saved as .nii.gz file in temporary folder: /var/folders/mz/n0dm07dn7r5g8lt_cv00nhpr0000gn/T/tmpjphwy1ef


# Running the Model on your Image Stack
The `nnUNetv2_predict` command calls the nnU-Net framework to predict the location of the structures in the image stack. Below are the details of the flags used in the command:

- `-i`: Specifies the input folder containing the image stack.
- `-o`: Specifies the output folder to save the results.
- `-d`: Specifies the dataset ID (provided).
- `-c`: Specifies the resolution of the model.
- `--save_probabilities`: Saves the predicted probabilities for each class.

In [37]:
import os
import shutil

# Dynamically find the path to nnUNetv2_predict
home_dir = os.path.expanduser("~")
local_bin_dir = os.path.join(home_dir, ".local", "bin")
nnunet_path = shutil.which("nnUNetv2_predict", path=local_bin_dir)

if nnunet_path is None:
    raise FileNotFoundError("nnUNetv2_predict not found in ~/.local/bin. Ensure it is installed and accessible.")

# Example usage
input_folder = os.environ.get('INPUT_FOLDER')
output_folder = "outputFolder"

# Run the command
os.system(f"{nnunet_path} -i {input_folder} -o {output_folder} -d 123 -c 3d_fullres --save_probabilities")



#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting amos:
perform_everything_on_device: True


100%|██████████| 4/4 [00:01<00:00,  3.35it/s]


sending off prediction to background worker for resampling and export
done with amos


0

In [38]:
import os
import nibabel as nib
import numpy as np
import json

# Define the output folder and find the .nii.gz file
nii_files = [f for f in os.listdir(output_folder) if f.endswith('.nii.gz')]
if not nii_files:
    raise FileNotFoundError("No .nii.gz files found in the output folder.")
file_path = os.path.join(output_folder, nii_files[0])

# Load the .nii.gz file
img = nib.load(file_path)

# Convert the image data to a NumPy array
image_data = img.get_fdata()

# Get unique region numbers, excluding background (0) and numbers greater than 12
unique_regions = np.unique(image_data)
filtered_regions = unique_regions[(unique_regions > 0) & (unique_regions <= 12)]
print(filtered_regions)
# Look up the plan2_path.json file in the folder specified by the nnUNet_raw environment variable
nnUNet_raw = os.environ.get('nnUNet_raw')
nnUNet_raw = os.path.join(nnUNet_raw, "Dataset123_Foo", "nnUNetTrainer__nnUNetPlans__3d_fullres")
if not nnUNet_raw:
    raise EnvironmentError("Environment variable 'nnUNet_raw' is not set.")
plan2_path = os.path.join(nnUNet_raw, "plan2.json")
if not os.path.exists(plan2_path):
    raise FileNotFoundError(f"plan2_path.json not found in {nnUNet_raw}.")

# Load the plan2_path.json file
with open(plan2_path, 'r') as f:
    plan2_data = json.load(f)

# Print the strings corresponding to the region numbers
for region in filtered_regions:
    region_str = plan2_data.get(str(int(region)), "Unknown region")
    print(f"Region {int(region)}: {region_str}")

[]
